In [1]:
import sys
import os
import numpy as np
import pandas as pd

repo_path =  r"C:\Users\tebogo.mojela\Documents\GitHub\Value-Frameworks-PDF-Lifting"
microservices_path = r"C:\Users\tebogo.mojela\Documents\GitHub\Value-Frameworks-PDF-Lifting\Microservices"

## Microservice Imports

In [2]:
sys.path.append(os.path.abspath(microservices_path))
import tk_pandas_essentials
import tk_pdf_parser
#tk_pandas_essentials.desc()
tk_pandas_essentials.unwrap_row_items()

## CSV Exploration

In [3]:
df_pdf_names = pd.read_csv(repo_path+"/Input Data/PDFnames.csv", )
df_amkey_goldstandard = pd.read_csv(repo_path+"/Input Data/AMKEY_GoldenStandard.csv", )
df_amkey_synonyms = pd.read_csv(repo_path+"/Input Data/ActivityMetricsSynonyms.csv", )
df_train = pd.read_csv(repo_path+"/Input Data/Train.csv", )
df_sample_submission = pd.read_csv(repo_path+"/Input Data/SampleSubmission.csv", )

### PDFnames

In [4]:
tk_pandas_essentials.desc_df(df_pdf_names)

Shape: (12, 2)
Columns: Index(['ID', 'File Name'], dtype='object')




,ID,File Name
0,Oceana1.pdf,Oceana_Group_Sustainability_Report_2022.pdf
1,Ssw.pdf,ssw-IR22.pdf
2,Sasol.pdf,SASOL Sustainability Report 2023 20-09_0.pdf
3,Impala.pdf,ESG-spreads.pdf
4,Clicks.pdf,Clicks-Sustainability-Report-2022.pdf


### AMKEY_GoldenStandard
- at least one company reported
- unique

In [5]:
tk_pandas_essentials.desc_df(df_amkey_goldstandard)

Shape: (511, 2)
Columns: Index(['AMKEY', 'ActivityMetric'], dtype='object')




,AMKEY,ActivityMetric
0,3,Advisory fees as per income statement
1,6,Air emissions of the following pollutants: (1) CO
2,7,Air emissions of the following pollutants: (2) NOx (excluding N2O)
3,8,Air emissions of the following pollutants: (3) SOx
4,9,Air emissions of the following pollutants: (4) particulate matter (PM10)


In [6]:
len(set(df_amkey_goldstandard["AMKEY"]))

511

### ActivityMetricsSynonyms

In [7]:
tk_pandas_essentials.desc_df(df_amkey_synonyms)

Shape: (365, 4)
Columns: Index(['AMKEY', 'Group', 'ActivityMetric', 'ClientMetric'], dtype='object')




,AMKEY,Group,ActivityMetric,ClientMetric
0,7,Impala,Air emissions of the following pollutants: (2) NOx (excluding N2O),Total indirect Nox
1,7,Sasol,Air emissions of the following pollutants: (2) NOx (excluding N2O),Nitrogen oxides (NOx) (kilotons)
2,8,Sasol,Air emissions of the following pollutants: (3) SOx,Sulphur oxides (SOx ) (kilotons)
3,8,Ssw,Air emissions of the following pollutants: (3) SOx,SO2 emissions
4,8,Impala,Air emissions of the following pollutants: (3) SOx,Total direct SO2 + Total indirect SO2


In [8]:
len(set(df_amkey_synonyms["Group"]))

8

In [9]:
set(df_amkey_synonyms["Group"])

{'Absa', 'Distell', 'Impala', 'Oceana1&2', 'Sasol', 'Ssw', 'Tongaat', 'Uct1&2'}

In [10]:
len(set(df_amkey_synonyms["AMKEY"]))

156

### Train
- no value = not reported
- AMKEY_X_Company = ID

In [11]:
tk_pandas_essentials.desc_df(df_train)

Shape: (5110, 4)
Columns: Index(['ID', '2021_Value', '2020_Value', '2019_Value'], dtype='object')




,ID,2021_Value,2020_Value,2019_Value
0,3_X_Absa,NaN,NaN,NaN
1,6_X_Absa,NaN,NaN,NaN
2,7_X_Absa,NaN,NaN,NaN
3,8_X_Absa,NaN,NaN,NaN
4,9_X_Absa,NaN,NaN,NaN


In [12]:
df_train['2021_Value'] = df_train['2021_Value'].replace('Not tracked', np.nan)

In [13]:
df_train_real_values = tk_pandas_essentials.cut_nulls(df_train, "2021_Value")

Before: (5110, 4)
After: (460, 4)
Trashed: 4650


In [14]:
tk_pandas_essentials.desc_df(df_train_real_values)

Shape: (460, 4)
Columns: Index(['ID', '2021_Value', '2020_Value', '2019_Value'], dtype='object')




,ID,2021_Value,2020_Value,2019_Value
27,46_X_Absa,4400000000.0,3500000000.0,2.700000e+09
30,49_X_Absa,1.0,2.0,1.000000e+00
33,52_X_Absa,98.0,99.0,9.500000e+01
34,53_X_Absa,3.0,3.0,4.000000e+00
35,54_X_Absa,12.0,13.0,1.200000e+01


In [15]:
def is_numeric(value):
    try:
        pd.to_numeric(value)
        return True
    except ValueError:
        print(value)
        return False
    
all_numeric = df_train_real_values[["2021_Value", "2020_Value", "2019_Value"]].apply(lambda col: col.apply(is_numeric)).all()

print(all_numeric)

2021_Value    True
2020_Value    True
2019_Value    True
dtype: bool


### SampleSubmission

In [16]:
tk_pandas_essentials.desc_df(df_sample_submission)

Shape: (5110, 2)
Columns: Index(['ID', '2022_Value'], dtype='object')




,ID,2022_Value
0,100_X_Absa,0.0
1,100_X_Clicks,0.0
2,100_X_Distell,0.0
3,100_X_Impala,0.0
4,100_X_Oceana1&2,0.0


# Testing & Debugging

In [17]:
pdf_path = repo_path+"/Input Data/Data Sources/Structured data/2022-Absa-Group-limited-Environmental-Social-and-Governance-Data-sheet.pdf"
text_content, tables = tk_pdf_parser.extract_text_and_tables(pdf_path)

In [18]:
text_content_lw = text_content.lower()

In [19]:
present_keys = []
for index, item in df_amkey_synonyms["ActivityMetric"].iteritems():
    if item.lower() in text_content_lw:
        key = df_amkey_synonyms["AMKEY"].iloc[index]
        client_metric = df_amkey_synonyms["ClientMetric"].iloc[index]
        present_keys.append({"AM": item.lower(), "CM": str(client_metric).lower(), "KEY": str(key)})
        
for index, item in df_amkey_synonyms["ClientMetric"].iteritems():
    if item.lower() in text_content_lw:
        key = df_amkey_synonyms["AMKEY"].iloc[index]
        act_metric = df_amkey_synonyms["ActivityMetric"].iloc[index]
        present_keys.append({"AM":str(act_metric).lower(),"CM": item.lower(),"KEY": str(key)})

In [20]:
df_present_keys = pd.DataFrame(present_keys)
df_present_keys.head()

,AM,CM,KEY
0,revenue,turnover,385
1,revenue,revenue,385
2,revenue,recurrent revenue,385
3,revenue,rand value of total revenue generated in south africa,385
4,total number of employees,no. of employees including contractors – total,523


In [21]:
absa_training_data = tk_pdf_parser.tabulate(pdf_path, "Absa.pdf", "2022-Absa-Group-limited-Environmental-Social-and-Governance-Data-sheet.pdf", "Absa")

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [22]:
len(absa_training_data)

2571

In [23]:
import re
import json

json_data = json.dumps(absa_training_data, indent=2)
print(json_data)

[
  {
    "document_id": "Absa.pdf",
    "document_name": "2022-Absa-Group-limited-Environmental-Social-and-Governance-Data-sheet.pdf",
    "group": "Absa",
    "activity_metrics": {
      "activity_metric": "Number of Equator Principle transactions",
      "year": "2022",
      "stat_value": "6"
    }
  },
  {
    "document_id": "Absa.pdf",
    "document_name": "2022-Absa-Group-limited-Environmental-Social-and-Governance-Data-sheet.pdf",
    "group": "Absa",
    "activity_metrics": {
      "activity_metric": "Number of Equator Principle transactions",
      "year": "2021",
      "stat_value": "2"
    }
  },
  {
    "document_id": "Absa.pdf",
    "document_name": "2022-Absa-Group-limited-Environmental-Social-and-Governance-Data-sheet.pdf",
    "group": "Absa",
    "activity_metrics": {
      "activity_metric": "Number of Equator Principle transactions",
      "year": "2020",
      "stat_value": "3"
    }
  },
  {
    "document_id": "Absa.pdf",
    "document_name": "2022-Absa-Group-limi

### Look at again

In [24]:
# import re
# import json

# text_data = text_content.lower()

# year_pattern = r'\b\d{4}\b'
# numerical_value_pattern = r'\d+(?:,\d+)?(?:\.\d+)?'

# # Define activity metrics
# activity_metrics = list(set(df_present_keys["AM"]))

# # Extract relevant information from the text
# collection = {"document_id": "abc",
#         "document_name": "xyz",
#         "activity_metrics": {}}

# for metric in activity_metrics:
#     for match in re.finditer(metric + r'\s*' + year_pattern + r'\s*' + numerical_value_pattern, text_data):
#         year = match.group(0)  # Entire match is the year
#         value = match.group(1)  # Capture group 1 contains the value
#         collection.activity_metrics = {
#             "activity_metric": metric,
#             "year": year,
#             "statistical_value": value
#         }

# # Convert data to JSON format
# json_data = json.dumps(collection, indent=2)
# print(json_data)

In [25]:
tk_pandas_essentials.search_dfa(795, df_amkey_synonyms, "AMKEY")

Message: SEARCH_DFB (slightly slower than SEARCH_DFA): Make sure the item you are searching for is of type STRING and the col is of datatype OBJECT(string)


,AMKEY,Group,ActivityMetric,ClientMetric
321,795,Ssw,Number of independent board members,Independent non-executive directors
322,795,Absa,Number of independent board members,Independent non-executive directors
323,795,Tongaat,Number of independent board members,Independent Board members
324,795,Distell,Number of independent board members,Number of board members who are deemed 'independent'


In [26]:
search_string ="795_X_Absa"
tk_pandas_essentials.search_dfa(search_string, df_train, "ID")

,ID,2021_Value,2020_Value,2019_Value
367,795_X_Absa,12.0,11.0,12.0


In [27]:
present_key_binds = []
for key_string in present_keys:
    key = key_string.split("_")[-1]
    key_string = key_string.split("_")[0]
    print("Key_String: "+key_string)
    print("Key: "+key)
    act_metric = tk_pandas_essentials.search_dfa(key_string, df_amkey_synonyms, "ActivityMetric")
    client_metric = tk_pandas_essentials.search_dfa(key_string, df_amkey_synonyms, "ClientMetric")
    alias_pair = {}
    
    tk_pandas_essentials.desc_df(act_metric)
    print("\n")
    tk_pandas_essentials.desc_df(client_metric)
    
    tk_pandas_essentials.stop()
    if (tk_pandas_essentials.is_null(act_metric) == True and tk_pandas_essentials.is_null(client_metric) == False):
        alias_pair = {
            "AM": client_metric["ActivityMetric"].iloc[0],
            "CM": key_string
            }
    elif (tk_pandas_essentials.is_null(act_metric) == False and tk_pandas_essentials.is_null(client_metric) == True):
        alias_pair = {
            "AM": key_string,
            "CM": act_metric["ClientMetric"].iloc[0]
            }
        
    elif (tk_pandas_essentials.is_null(act_metric) == False and tk_pandas_essentials.is_null(client_metric) == False):
        alias_pair = {
            "AM": act_metric["ActivityMetric"].iloc[0],
            "CM": client_metric["ClientMetric"].iloc[0]
            }
    present_key_binds.append(alias_pair) 

AttributeError: 'dict' object has no attribute 'split'

In [ ]:
present_key_binds

In [ ]:
key = ""
for index,row in df_train["ID", "2021_Value"].iterrows():
    key = tk_pandas_essentials.search_dfa(item, df_amkey_goldstandard, "ActivityMetric")
    # if key.lower() in text_content_lw:
    #     present_keys.append(key.lower())

In [ ]:
df_train

In [ ]:
df_amkey_goldstandard

In [ ]:
set(items)

In [ ]:
len(set(df_sample_submission["ID"]))